# Memory (대화 내용 기억)

- 대화 내용 기억
- LLM은 기본적으로 대화 내용을 기억하지 않는다(stateless)
- 이전 대화내용을 계속 프롬포트에 주입해야함

1. Short-Term Memory
    - 단기기억: 한 대화 세션에 대한 기억
2. Long-Term Memory
    - 장기기억: 전체 세션에서 추출한 중요한 정보


## Memory 구동 방식
1. 메모리는 기본적으로 모든 대화 내역을 LLM Input에 밀어넣는 것.
2. 이때 대화가 길어지면 토큰수 증가 및 성능 하락이 일어남
3. 개선 방식 컨셉
    1. 요약
    2. 적정 길이에서 앞부분 자르기
    3. 정리(특정 명사들로 정리, Node-Edge 그래프 방식 등)



## `ConversationBufferMemory`
- 메시지 저장 -> 변수에서 추출 가능

In [3]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리를 저장할 변수는 {}다. 기존에 대화내용이 있다면 불러와라
memory.load_memory_variables({})

{'chat_history': []}

In [4]:
from operator import itemgetter
# `chat_history`변수에, load_memory_var 결과를 저장하고, `chat_history 키를 추출`
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

In [5]:
chain = runnable | prompt | llm 

In [6]:
res = chain.invoke({'input': '만나서 반가워'})

In [7]:
memory.save_context(
    {'human': '만나서 반가워'},
    {'ai': res.content}
)

In [8]:
my_message = '내 이름 뭐라고?'
res = chain.invoke({'input': my_message})
memory.save_context(
    {'human': my_message},
    {'ai': res.content}
)

In [9]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반가워', additional_kwargs={}, response_metadata={}),
  AIMessage(content='저도 만나서 반가워요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='내 이름 뭐라고?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='죄송하지만, 제가 당신의 이름을 알지 못해요. 이름을 알려주시면 기억해둘게요!', additional_kwargs={}, response_metadata={})]}

In [11]:
import time
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)


# Prompt
prompt = ChatPromptTemplate(
    [
        ('system', '넌 좀 틱틱대는 챗봇이야. '),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

# Memory
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

chain = runnable | prompt | llm | StrOutputParser()

# 메세지 스트리밍 하는 함수
def stream_msg(chain, msg):
    full_msg = ''
    print('AI: ', end='')
    for token in chain.stream({'input': msg}):
        full_msg += token
        print(token, end='', flush=True)
    print('\n---')
    return full_msg

input_msg = ''

# 사용자가 ('quit', '정지', '그만') 중에 하나를 입력하면 대화 종료
while input_msg not in ('quit', '정지', '그만'):
    input_msg = input()
    print('인간: ', input_msg)
    output_msg = stream_msg(chain, input_msg)
    memory.save_context(
        {'human': input_msg},
        {'ai': output_msg}
    )
    time.sleep(0.5)

인간:  안녕
AI: 안녕? 뭐 궁금한 거 있어?
---
인간:  
AI: 뭐, 말 좀 해봐. 궁금한 거 있으면 알려줘.
---
인간:  정지
AI: 뭐? 정지라고? 그냥 가만히 있으라는 건가? 좀 더 명확하게 말해봐.
---


In [ ]:
# 레포트 작성 챗봇
from operator import itemgetter
from pprint import pprint
from pydantic import BaseModel, Field

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

# Output parser 정의
class Report(BaseModel):
    title: str = Field(..., description='보고서의 제목')
    summary: str = Field(..., description='보고서 요약본')
    content_kr: str = Field(..., description='한국어로 작성된 보고서의 내용(1000자 이내)')
    content_jp: str = Field(..., description='일본어로 작성된 보고서의 내용(1000자 이내)')

parser = PydanticOutputParser(pydantic_object=Report)

# Prompt에 format instructions 추가
prompt = ChatPromptTemplate(
    [
        ('system', '넌 보고서 작성에 특화된 챗봇이야. '
                   '반드시 지정된 형식에 맞춰 JSON으로 답변해.\n\nFORMAT INSTRUCTION: {format_instructions}'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}'),
    ]
).partial(format_instructions=parser.get_format_instructions())

# Memory
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

# 체인 구성
chain = runnable | prompt | llm | parser

input_msg = ''

# 사용자가 ('quit', '정지', '그만', '') 중에 하나를 입력하면 대화 종료
while 1:
    input_msg = input()
    if input_msg  in ('quit', '정지', '그만', ''):
        break

    print('인간: ', input_msg)
    output_msg = chain.invoke({'input': input_msg})
    pprint(output_msg.model_dump())
    memory.save_context(
        {'human': input_msg},
        {'ai': output_msg.model_dump_json()}
    )